# Cantilever beam

This is a continuation of the example_2, where MeshPy is used to create the input file for 4C.

### Creating Cantilever

Inorder to create a cantilever for analysis we require element type(meshpy.four_c.element_beam), material (meshpy.four_c.material) and function for the shape (meshpy.mesh_creation_functions). also need mesh object to hold all this attributes of cantilever.

In [ ]:
from meshpy.four_c.element_beam import Beam3rHerm2Line3, Beam3rLine2Line2
from meshpy.four_c.input_file import InputFile
from meshpy.four_c.material import MaterialReissner
from meshpy.mesh_creation_functions.beam_basic_geometry import (
    create_beam_mesh_arc_segment_via_axis,
    create_beam_mesh_line,
)

input_file = InputFile()
beam_type = Beam3rHerm2Line3
beam_material = MaterialReissner(radius=0.1, youngs_modulus=1.0)

In [ ]:
cantilever_arc = create_beam_mesh_arc_segment_via_axis(
    input_file,
    beam_type,
    beam_material,
    [0, 1, 0],
    [0, 0, -10],
    [0, 0, 0],
    0.785,
    n_el=10,
)

Cantilever object consist of set of nodes based on the function used to create the shape, using beam meash line create set of 3 geometry objects.

In [ ]:
# print(cantilever)

In [ ]:
import pyvista as pv

# pv.set_jupyter_backend('tramstate')  # or 'trame' but doesnot work
input_file.display_pyvista()

### Define the boundary conditions

In [ ]:
from meshpy.core.conf import mpy
from meshpy.four_c.boundary_condition import BoundaryCondition
from meshpy.four_c.function import Function

ft = Function("SYMBOLIC_FUNCTION_OF_TIME t")
fxt = Function("SYMBOLIC_FUNCTION_OF_SPACE_TIME t")
input_file.add(ft, fxt)
input_file.add(
    BoundaryCondition(
        cantilever_arc["start"],
        "NUMDOF 9 ONOFF 1 1 1 1 1 1 0 0 0 VAL 1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 FUNCT 0 0 0 0 0 0 0 0 0",
        bc_type=mpy.bc.dirichlet,
    )
)
input_file.add(
    BoundaryCondition(
        cantilever_arc["end"],
        "NUMDOF 9 ONOFF 0 0 1 0 0 0 0 0 0 VAL 0.0 0.0 -0.000001 0.0 0.0 0.0 0.0 0.0 0.0 FUNCT 0 0 1 0 0 0 0 0 0",
        bc_type=mpy.bc.neumann,
    )
)

### Define the Simulation parameter

In [ ]:
from meshpy.four_c.header_functions import set_header_static, set_runtime_output

set_header_static(input_file, time_step=0.1, n_steps=2)
set_runtime_output(input_file)

### Create the input file 

In [ ]:
input_file.write_input_file("./cantilever_arc/cantilever.dat")

### Running 4C Simulation with meshpy

In [ ]:
from meshpy.four_c.run_four_c import run_four_c

run = run_four_c(
    input_file="/home/a11basna/meshpy/examples/cantilever_arc/cantilever.dat",
    output_dir="./cantilever_arc/",
    four_c_exe="/home/a11basna/4C/build/4C",
)
assert run == 0

### Visualizing the results

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("trame")  # or 'trame' but doesnot work

reader = pv.get_reader("./cantilever_arc/xxx-structure-beams.pvd")
reader.time_values
reader.set_active_time_point(2)
mesh = reader.read()[0]

# Plot in x–z plane (looking along -y)
plotter = pv.Plotter()
plotter.add_mesh(mesh, color="red", show_edges=True)
plotter.view_vector((0, -1, 0))
plotter.show()